## 데이터편향이 예측에 미치는 영향 - 타이타닉호 생존여부 머신러닝(SVM)으로 예측하기

### 1. 사용 패키지와 모듈 임포트

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split #학습데이터와 테스트데이터 분할
from sklearn import svm      #SVM 모델
from sklearn import metrics  #정확도 비교

import seaborn as sns

pd.set_option('mode.chained_assignment',  None) # Warning 방지용

### 2. 데이터 가져오기

In [ ]:
df = pd.read_csv('./titanic.csv')

data_df = df             # 데이터 모두 가져오는 경우
data_df.head()

In [ ]:
#pclass : Passenger Class, 승객등급
#name : 승객이름
#sex : 승객성별 (여성은 0, 남성은 1)
#age : 승객나이
#sibsp : 동승한 형제 또는 배우자 수
#parch : 동승한 부모 또는 자녀 수
#ticket : 티켓번호
#fare : 승객 지불 요금
#cabin : 선실이름
#embarked : 승선항(C=쉘 브르크, Q=퀸즈타운, S=사우스 햄튼)
#survived : 생존 여부(생존은 1, 아닌경우는 0)

In [ ]:
#데이터의 행,열 개수 확인
data_df.shape  

In [ ]:
#나이 데이터를 범주화 하기 위해서 10으로 나눈 몫에서 10을 곱해줌 
data_df['age'] = (data_df['age']//10) * 10  
data_df.head()

In [ ]:
#생존자 1, 사망자 0, 학습데이터에서 각각 수를 새어 막데 그래프로 나타내기

print(data_df['survived'].value_counts())
data_df['survived'].value_counts().plot(kind='bar')

# 전체 데이터(1046명)에서 사망율은 59%(619명), 생존율은 41%(427명)

### 3. 데이터셋 나누기

In [ ]:
#학습데이터와 테스트 데이터를 자동으로 나눠주는 함수

train, test = train_test_split(data_df, test_size= 0.2) #일반적으로 80%/20%, 70%/30% 비율로 분할
print("train data", train.shape)
print("test data",test.shape)

### 4. 학습에 사용할 변수(특징, Feature) 선택하기
#### - 온라인 강의 실습에서는 가장 정확도가 우수하게 나온 변수는 pclass와 sex 이다.

In [ ]:
#데이터 타입, 각 아이템 개수, 누락데이터(NaN(Not a Number or Not Available) 등 확인
#누락데이터가 많이 없는 피처를 선택
data_df.info()   

In [ ]:
data_df.corr()   # 상관계수 보기

In [ ]:
train.head()

In [ ]:
test.head()

#### 4-1. 학습데이터에서 각 피쳐(Feature(특징데이터), 변수)가 생존자 예측 분류에 미치는 영향 탐색

In [ ]:
#생존자 1, 사망자 0, 학습데이터에서 각각 수를 세어 막데 그래프로 나타내기

print(train['survived'].value_counts())
train['survived'].value_counts().plot(kind='bar')

#전체 학습데이터(836명)에서 사망자는 60%(497명), 생존자는 40%(339명)

In [ ]:
#학습데이터의 pclass 피처에서 survived 피처를 기준으로 그룹을 나누어 그룹별 분포를 살펴봄
#1은 생존자 0은 사망자

print(train['pclass'].value_counts()) # plclass내에서 각 그룹별 수를 세어줌
ax = sns.countplot(x='pclass', hue= 'survived', data= train)  # pclass의 count의 수를 hue에 설정된 카테고리로 나눠 그려줌

#1등급 클래스의 생존자가 가장 많고, 3등 클래스의 사망자가 가장 많음

In [ ]:
#학습데이터의 survived 피처를 기준으로 그룹을 나누어 그룹별 sex 피처의 분포를 살펴봄 (0은 여성, 1은 남성)
print(train['sex'].value_counts())
ax = sns.countplot(x='sex', hue= 'survived', data= train)

#여성 생존자의 생존율이 월등히 높음

In [ ]:
#학습데이터의 survived 피처를 기준으로 그룹을 나누어 그룹별 age 피처의 분포를 살펴봄 (0은 여성, 1은 남성)
print(train['age'].value_counts())
ax = sns.countplot(x='age', hue= 'survived', data= train)

#20,30대 탑승객이 가장 많고 가장 많이 사망

In [ ]:
train.groupby(['pclass','sex']).mean()
#1등급 탑승객은 평균연령이 가장 높음, 1등급 클래스 여성이 가장 생존율이 높음, 2,3등급 클래스 남성이 가장 생존율이 낮음

In [ ]:
train.groupby(['pclass','age']).mean()
#연령대별로는 영아와 10대 생존율이 가장 높음

### 위의 데이터에 따르면 생존율에 가장 많은 영향을 주는 변수는 pclass와 sex임을 알 수 있다.
### pclass와 sex 변수만을 선택해서 학습 모델을 만들어 보자.

### 5. 학습용 데이터셋 : 학습데이터와 레이블(정답) 나누기

In [ ]:
train_data_df = train[['pclass','sex']]
train_data_df 

In [ ]:
train_label_df = train[['survived']]
train_label_df

In [ ]:
train_data = train_data_df.values           # 값부분만 가져오기(2차원 형태)
train_data

In [ ]:
train_label = train_label_df.values.ravel() # 값부분을 1차원으로 펴주기
train_label

### 6. 테스트 데이터셋 : 학습데이터와 레이블(정답) 나누기

In [ ]:
test_data_df = test[['pclass','sex']]
test_data_df

In [ ]:
test_label_df = test[['survived']]  
test_label_df

In [ ]:
test_data = test_data_df.values           # 값부분만 가져오기(2차원 형태)
test_data

In [ ]:
test_label = test_label_df.values.ravel() # 값부분을 1차원으로 펴주기
test_label

### 7. SVM 모델 학습하기

In [ ]:
clf = svm.SVC(C = 1, gamma = 0.1)     #svm 객체 생성
clf.fit(train_data, train_label)      #svm 모델로 학습데이터로 학습하기

### 8. 테스트 데이터로 예측하기

In [ ]:
pred_svm = clf.predict(test_data)
pred_svm  #svm이 예측한 생존 여부 값

In [ ]:
#예측 결과를 test데이터에 추가해줌
test['survived_predict']=pred_svm
test

### 9. 테스트 데이터의 예측 결과 그래프로 나타내보기

In [ ]:
#테스트 데이터의 pclass 피처에서 survived_predict 피처를 기준으로 그룹을 나누어 그룹별 분포를 살펴봄
# 1은 생존자 0은 사망자

print(test['pclass'].value_counts()) # plclass내에서 각 그룹별 수를 세어줌
ax = sns.countplot(x='pclass', hue= 'survived_predict', data= test)  # pclass의 count의 수를 hue에 설정된 카테고리로 나눠 그려줌

#3등 클래스의 사망가능성이 매우 높다고 예측

In [ ]:
#테스트데이터의 survived_predict 피처를 기준으로 그룹을 나누어 그룹별 sex 피처의 분포를 살펴봄 (0은 여성, 1은 남성)
print(test['sex'].value_counts())
ax = sns.countplot(x='sex', hue= 'survived_predict', data= test)

#여성 사망자 수가 0, 남성 사망 가능성이 매우 높다고 예측

In [ ]:
#학습데이터의 survived_predict 피처를 기준으로 그룹을 나누어 그룹별 age 피처의 분포를 살펴봄 (0은 여성, 1은 남성)
print(test['age'].value_counts())
ax = sns.countplot(x='age', hue= 'survived_predict', data= test)

#20,30대 탑승객이 가장 많고 가장 많이 사망할 것으로 예측

In [ ]:
test.groupby(['pclass','sex']).mean()
#각 클래스별 여성은 생존, 남성은 사망할 것으로 예측